# Building The Dim Tables



In [250]:
import pandas as pd

def get_uniques_and_indexing(file_name, col1, col2="", col3="", col4="",col5="",col6="",col7=""):
    """
  This function efficiently extracts unique rows based on up to four columns, sorts them, and assigns numerical indices.

  Args:
      file_name (str): The path to the CSV file.
      col1 (str): The first column name for duplicate removal.
      col2 (str): The second column name (optional) for duplicate removal and sorting.
      col3 (str, optional): The third column name (optional) for duplicate removal.
      col4 (str, optional): The fourth column name (optional) for duplicate removal.
      col5 (str, optional): The fifth column name (optional) for duplicate removal.

  Returns:
      pandas.DataFrame: A new DataFrame containing unique rows, sorted, and with a new "ID" column for indexing.
    """
    df = pd.read_csv(file_name, index_col=False)
  # Select relevant columns based on provided arguments
    selected_cols = [col1]
    if col2:
        selected_cols.append(col2)
    if col3:
        selected_cols.append(col3)
    if col4:
        selected_cols.append(col4)
    if col5:
        selected_cols.append(col5)
    if col6:
        selected_cols.append(col6)
    if col7:
        selected_cols.append(col7)
    df_new = df[selected_cols]
    # Remove duplicate rows based on col1 (and optionally col2,col3,col4,col5,col6)
    df_new_unique = df_new.drop_duplicates(subset=selected_cols)
    # Sort and add "ID" column
    df_new_sorted = df_new_unique.sort_values(by=selected_cols, ascending=True)
    df_new_sorted["ID"] = range(1, len(df_new_sorted) + 1)
    # Return desired columns
    return df_new_sorted[["ID"] + selected_cols].reset_index(drop=True)

# Use The Created Funcation to Create Dim_Tables

# Dim Store

In [419]:
Stores=get_uniques_and_indexing('EndInvFINAL12312016.csv','Store',"City")
Stores

,ID,Store,City
0,1,1,HARDERSFIELD
1,2,2,ASHBORNE
2,3,3,HORNSEY
3,4,4,EANVERNESS
4,5,5,SUTTON
...,...,...,...
75,76,76,DONCASTER
76,77,77,TAMWORTH
77,78,78,EASTHAVEN
78,79,79,BALLYMENA


In [420]:
Stores=Stores.sort_values(by='ID')

In [421]:
Stores

,ID,Store,City
0,1,1,HARDERSFIELD
1,2,2,ASHBORNE
2,3,3,HORNSEY
3,4,4,EANVERNESS
4,5,5,SUTTON
...,...,...,...
75,76,76,DONCASTER
76,77,77,TAMWORTH
77,78,78,EASTHAVEN
78,79,79,BALLYMENA


# Dim Supllier

In [5]:
Suppliers=get_uniques_and_indexing('InvoicePurchases12312016.csv',"VendorNumber","VendorName")

In [6]:
Suppliers

,ID,VendorNumber,VendorName
0,1,2,"IRA GOLDMAN AND WILLIAMS, LLP"
1,2,54,AAPER ALCOHOL & CHEMICAL CO
2,3,60,ADAMBA IMPORTS INTL INC
3,4,105,ALTAMAR BRANDS LLC
4,5,200,AMERICAN SPIRITS EXCHANGE
...,...,...,...
124,125,98450,Serralles Usa LLC
125,126,99166,STARK BREWING COMPANY
126,127,172662,SWEETWATER FARM
127,128,173357,TAMWORTH DISTILLING


# Dim_Brand

In [338]:
Brands=get_uniques_and_indexing('PurchasesFINAL12312016.csv',"Brand","VendorName","VendorNumber")

In [339]:
Brands

,ID,Brand,VendorName,VendorNumber
0,1,58,SHAW ROSS INT L IMP LTD,8320
1,2,60,JIM BEAM BRANDS COMPANY,12546
2,3,61,SAZERAC CO INC,8004
3,4,62,BROWN-FORMAN CORP,1128
4,5,63,BROWN-FORMAN CORP,1128
...,...,...,...,...
10688,10689,90090,ALISA CARR BEVERAGES,1703
10689,10690,90604,ALISA CARR BEVERAGES,1703
10690,10691,90609,FLAVOR ESSENCE INC,201359
10691,10692,90609,"IRA GOLDMAN AND WILLIAMS, LLP",2


In [340]:
df_raw=pd.merge(Brands, Suppliers, on=['VendorName',"VendorNumber"],how='left')
Brand=df_raw[['ID_x','Brand','ID_y']]
Brand.rename(columns={"ID_y":'SupplierID'},inplace=True)
Brand.rename(columns={'ID_x':'BrandID'},inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_9244\4108909657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brand.rename(columns={"ID_y":'SupplierID'},inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_9244\4108909657.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brand.rename(columns={'ID_x':'BrandID'},inplace=True)


In [341]:
Brand

,BrandID,Brand,SupplierID
0,1,58,75
1,2,60,99
2,3,61,72
3,4,62,13
4,5,63,13
...,...,...,...
10688,10689,90090,26
10689,10690,90604,26
10690,10691,90609,129
10691,10692,90609,1


# Dim Product


In [364]:
Products=get_uniques_and_indexing('PurchasesFINAL12312016.csv',"Brand","VendorNumber","VendorName","Description","Size","PurchasePrice","Classification")
Products.rename(columns={'ID': 'ProductID'}, inplace=True)

In [365]:
Products['Classification'] = Products['Classification'].apply(lambda x: "Alchloic" if x == 1 else "Non-Alchloic")

In [366]:
Products

,ProductID,Brand,VendorNumber,VendorName,Description,Size,PurchasePrice,Classification
0,1,58,8320,SHAW ROSS INT L IMP LTD,Gekkeikan Black & Gold Sake,750mL,9.28,Alchloic
1,2,60,12546,JIM BEAM BRANDS COMPANY,Canadian Club 1858 VAP,750mL,7.40,Alchloic
2,3,61,8004,SAZERAC CO INC,Margaritaville Silver,750mL,10.60,Alchloic
3,4,62,1128,BROWN-FORMAN CORP,Herradura Silver Tequila,750mL,28.67,Alchloic
4,5,63,1128,BROWN-FORMAN CORP,Herradura Reposado Tequila,750mL,30.46,Alchloic
...,...,...,...,...,...,...,...,...
10694,10695,90090,1703,ALISA CARR BEVERAGES,Ch Lafite Rothschild 12,750mL,448.27,Non-Alchloic
10695,10696,90604,1703,ALISA CARR BEVERAGES,Ch Lynch Bages Pauilac,750mL,78.42,Non-Alchloic
10696,10697,90609,2,"IRA GOLDMAN AND WILLIAMS, LLP",Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic
10697,10698,90609,201359,FLAVOR ESSENCE INC,Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic


*Based On Our Scheme We Need To Make Adjustment For our Products Data Frame*

1. *We Need To Get The Sale Price Of Each  Produt it present in the sales Transacion Csv*

In [367]:
product_sold=pd.read_csv("SalesFINAL12312016.csv")
Products_merged=pd.merge(Products,product_sold,on=["Brand","Description","Size","VendorName","VendorNumber"],how='left')
Products_merged=Products_merged[["ProductID","SalesPrice"]].drop_duplicates()
Products=pd.merge(Products,Products_merged,on=["ProductID"],how='left')

In [369]:
Products.drop_duplicates(subset='ProductID',inplace=True)

In [370]:
Products

,ProductID,Brand,VendorNumber,VendorName,Description,Size,PurchasePrice,Classification,SalesPrice
0,1,58,8320,SHAW ROSS INT L IMP LTD,Gekkeikan Black & Gold Sake,750mL,9.28,Alchloic,12.99
1,2,60,12546,JIM BEAM BRANDS COMPANY,Canadian Club 1858 VAP,750mL,7.40,Alchloic,10.99
3,3,61,8004,SAZERAC CO INC,Margaritaville Silver,750mL,10.60,Alchloic,13.99
4,4,62,1128,BROWN-FORMAN CORP,Herradura Silver Tequila,750mL,28.67,Alchloic,41.99
6,5,63,1128,BROWN-FORMAN CORP,Herradura Reposado Tequila,750mL,30.46,Alchloic,43.99
...,...,...,...,...,...,...,...,...,...
14001,10695,90090,1703,ALISA CARR BEVERAGES,Ch Lafite Rothschild 12,750mL,448.27,Non-Alchloic,NaN
14002,10696,90604,1703,ALISA CARR BEVERAGES,Ch Lynch Bages Pauilac,750mL,78.42,Non-Alchloic,NaN
14003,10697,90609,2,"IRA GOLDMAN AND WILLIAMS, LLP",Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic,NaN
14004,10698,90609,201359,FLAVOR ESSENCE INC,Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic,NaN


2. *in order to Normalize our Products table We Need to join it with Suppliers & Brand ID*

In [371]:
Products=pd.merge(Products,Suppliers, on=['VendorName',"VendorNumber"])
Products.rename(columns={'ID':'SupplierID'},inplace=True)
Products=pd.merge(Products,Brand ,on=["Brand",'SupplierID'])
Products.rename(columns={'ID_x':'BrandID'},inplace=True)

In [372]:
Products

,ProductID,Brand,VendorNumber,VendorName,Description,Size,PurchasePrice,Classification,SalesPrice,SupplierID,BrandID
0,1,58,8320,SHAW ROSS INT L IMP LTD,Gekkeikan Black & Gold Sake,750mL,9.28,Alchloic,12.99,75,1
1,2,60,12546,JIM BEAM BRANDS COMPANY,Canadian Club 1858 VAP,750mL,7.40,Alchloic,10.99,99,2
2,3,61,8004,SAZERAC CO INC,Margaritaville Silver,750mL,10.60,Alchloic,13.99,72,3
3,4,62,1128,BROWN-FORMAN CORP,Herradura Silver Tequila,750mL,28.67,Alchloic,41.99,13,4
4,5,63,1128,BROWN-FORMAN CORP,Herradura Reposado Tequila,750mL,30.46,Alchloic,43.99,13,5
...,...,...,...,...,...,...,...,...,...,...,...
10694,10695,90090,1703,ALISA CARR BEVERAGES,Ch Lafite Rothschild 12,750mL,448.27,Non-Alchloic,NaN,26,10689
10695,10696,90604,1703,ALISA CARR BEVERAGES,Ch Lynch Bages Pauilac,750mL,78.42,Non-Alchloic,NaN,26,10690
10696,10697,90609,2,"IRA GOLDMAN AND WILLIAMS, LLP",Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic,NaN,1,10692
10697,10698,90609,201359,FLAVOR ESSENCE INC,Flavor Essence Variety 5 Pak,5/2 oz,17.00,Non-Alchloic,NaN,129,10691


3. *Ensure There is No Duplicates & put it on the required form*

In [373]:
Products.drop_duplicates(subset='ProductID',inplace=True)
Products = Products[["ProductID","Description","BrandID","SupplierID","Size","Classification","PurchasePrice","SalesPrice"]]

In [261]:
Products

,ProductID,Description,BrandID,SupplierID,Size,Classification,PurchasePrice,SalesPrice
0,1,Gekkeikan Black & Gold Sake,1,75,750mL,Alchloic,9.28,12.99
1,2,Canadian Club 1858 VAP,2,99,750mL,Alchloic,7.40,10.99
2,3,Margaritaville Silver,3,72,750mL,Alchloic,10.60,13.99
3,4,Herradura Silver Tequila,4,13,750mL,Alchloic,28.67,41.99
4,5,Herradura Reposado Tequila,5,13,750mL,Alchloic,30.46,43.99
...,...,...,...,...,...,...,...,...
10694,10695,Ch Lafite Rothschild 12,10689,26,750mL,Non-Alchloic,448.27,NaN
10695,10696,Ch Lynch Bages Pauilac,10690,26,750mL,Non-Alchloic,78.42,NaN
10696,10697,Flavor Essence Variety 5 Pak,10692,1,5/2 oz,Non-Alchloic,17.00,NaN
10697,10698,Flavor Essence Variety 5 Pak,10691,129,5/2 oz,Non-Alchloic,17.00,NaN


4. *Insert Data Into the Table*

# Make the Dim Time Table

*The Idea Is to Collect  Various Dates Across Diffrent CSV Files Then Create The Dim_Time Table*

In [374]:
import pandas as pd 

def get_unique_Dates(file_paths,date_columns):
    """
  This function efficiently extracts unique Dates From Varuios files With each one Has More Than One Dates Column .
   And Best Of that with Diffrent dates formate

  Args:
      file_paths (list):A list Contains The path to the CSV files.
      date_columns (list):A Nested List contain A lists of Date Columns with Date Columns in Eaach CSV file Respectively
  Returns:
      pandas.DataFrame: A new DataFrame containing unique rows, sorted, and with a new "ID" column for indexing.
    """
    all_dates = []
    # Extract dates from each file and column
    for file_path, columns in zip(file_paths, date_columns):
        df = pd.read_csv(file_path, usecols=columns)
        for column in columns:
            all_dates.extend(df[column].dropna().tolist())
    unique_dates = set(all_dates)
    sorted_unique_dates = list(unique_dates)
    # Create a DataFrame with the unique, sorted dates
    dates_df = pd.DataFrame({'Date': sorted_unique_dates})
    dates_df['Date'] = pd.to_datetime(dates_df['Date'],format='mixed')
    # Sort the DataFrame by the Date column (should already be sorted)
    dates_df = dates_df.sort_values(by='Date').reset_index(drop=True)
    dates_df['ID'] = range(1, len(dates_df) + 1)
    dates_df.drop_duplicates(subset="Date",inplace=True)
    dates_df=dates_df[['ID','Date']]
    dates_df['DayOfWeek']=dates_df['Date'].dt.day_name()
    dates_df['Month']=dates_df['Date'].dt.month
    dates_df['Quarter']=dates_df['Date'].dt.quarter
    dates_df['Year']=dates_df['Date'].dt.year
    dates_df["IsWeekend"] = dates_df['Date'].dt.day_of_week > 5
    return dates_df[['ID', 'Date', 'DayOfWeek', 'Month', 'Quarter', 'Year', 'IsWeekend']]

In [375]:
file_paths = [
    'PurchasesFINAL12312016.csv',
    'InvoicePurchases12312016.csv',
    'SalesFINAL12312016.csv'
]
# List of date columns in each file
date_columns =[
    ["PODate","ReceivingDate","InvoiceDate","PayDate"],  # Columns for file1.csv
    ['InvoiceDate',	'PODate','PayDate'],  # Columns for file2.csv
    ['SalesDate'] # Columns for file3.csv
    ]

In [ ]:
Time_df=get_unique_Dates(file_paths,date_columns)

In [377]:
Time_df

,ID,Date,DayOfWeek,Month,Quarter,Year,IsWeekend
0,1,2015-12-20,Sunday,12,4,2015,True
1,2,2015-12-21,Monday,12,4,2015,False
2,3,2015-12-22,Tuesday,12,4,2015,False
3,4,2015-12-23,Wednesday,12,4,2015,False
4,5,2015-12-24,Thursday,12,4,2015,False
...,...,...,...,...,...,...,...
482,483,2017-02-15,Wednesday,2,1,2017,False
483,484,2017-02-16,Thursday,2,1,2017,False
484,485,2017-02-17,Friday,2,1,2017,False
485,486,2017-02-18,Saturday,2,1,2017,False


# Star Building The Facts Tables 

*In Order to Normalize the Fact Tables*


1. *Start By Normalizing the Date Columns across the diffrent CSV files*

In [23]:
def normalized_dated_df(file_paths, date_columns):
    """ 
    This function normalizes date columns in multiple CSV files by replacing each date with a corresponding ID from a unified date dimension table.
    It ensures that all date columns are consistently represented across different files.
    And Save the new Data Framee into CSV After Being Normalized
    Args:
        file_paths (list): A list containing the paths to the CSV files.
        date_columns (list): A nested list containing lists of date columns in each CSV file respectively.
    Returns:
        dict: A dictionary where keys are file paths and values are pandas DataFrames with normalized date columns.
    """
    # Create the Time_df containing unique dates and their IDs
    Time_df = get_unique_Dates(file_paths, date_columns) #I Already defined it
    Time_df.to_csv("Dim_Time_keys.csv",index=False)
    
    normalized_dataframes = {}
    for file_path, columns in zip(file_paths, date_columns):
        df = pd.read_csv(file_path)
        df_normalized = df.copy()
        for col in columns:
            df_normalized[col] = pd.to_datetime(df_normalized[col], errors='coerce')
            Time_df['Date'] = pd.to_datetime(Time_df['Date'], errors='coerce')
            df_normalized = pd.merge(df_normalized, Time_df[['ID', 'Date']], left_on=col, right_on='Date', how='left')
            df_normalized[col] = df_normalized['ID']
            df_normalized.rename(columns={'ID':f'{col}_ID'},inplace=True)
            df_normalized.drop(columns=['Date',col], inplace=True)
        print(f"{file_path} is Succefully normalized and saved")
        normalized_dataframes[file_path] = df_normalized
    return normalized_dataframes

In [24]:
file_paths = [
    'PurchasesFINAL12312016.csv',
    'InvoicePurchases12312016.csv',
    'SalesFINAL12312016.csv'
]
# List of date columns in each file
date_columns =[
    ["PODate","ReceivingDate","InvoiceDate","PayDate"],  # Columns for file1.csv
    ['InvoiceDate',	'PODate','PayDate'],  # Columns for file2.csv
    ['SalesDate'] # Columns for file3.csv
    ]

In [25]:
normalized_dataframes = normalized_dated_df(file_paths, date_columns)

PurchasesFINAL12312016.csv is Succefully normalized and saved
InvoicePurchases12312016.csv is Succefully normalized and saved
SalesFINAL12312016.csv is Succefully normalized and saved


In [26]:
normalized_dataframes.keys()

dict_keys(['PurchasesFINAL12312016.csv', 'InvoicePurchases12312016.csv', 'SalesFINAL12312016.csv'])

# Fact Orders Table

In [27]:
Orders_df=normalized_dataframes['InvoicePurchases12312016.csv']

In [28]:
Orders_df

,VendorNumber,VendorName,PONumber,Quantity,Dollars,Freight,Approval,InvoiceDate_ID,PODate_ID,PayDate_ID
0,105,ALTAMAR BRANDS LLC,8124,6,214.26,3.47,NaN,18,2,104
1,4466,AMERICAN VINTAGE BEVERAGE,8137,15,140.55,8.57,NaN,24,3,114
2,388,ATLANTIC IMPORTING COMPANY,8169,5,106.60,4.61,NaN,28,5,104
3,480,BACARDI USA INC,8106,10100,137483.78,2935.20,NaN,34,1,82
4,516,BANFI PRODUCTS CORP,8170,1935,15527.25,429.20,NaN,24,5,96
...,...,...,...,...,...,...,...,...,...,...
5538,9622,WEIN BAUER INC,13626,90,1563.00,8.60,NaN,443,427,478
5539,9625,WESTERN SPIRITS BEVERAGE CO,13661,4617,37300.48,186.50,NaN,447,429,486
5540,3664,WILLIAM GRANT & SONS INC,13643,9848,202815.78,932.95,NaN,439,428,472
5541,9815,WINE GROUP INC,13602,24747,149007.56,819.54,NaN,440,426,476


In [29]:
Orders_df=pd.merge(Orders_df,Suppliers,on=['VendorName',"VendorNumber"],how='left')
Orders_df.drop('VendorName', axis=1, inplace=True)
Orders_df.drop('VendorNumber', axis=1, inplace=True)
Orders_df.drop('Approval', axis=1, inplace=True)
Orders_df.rename(columns={'ID':'SupplierID'},inplace=True)
Orders_df.rename(columns={'Dollars':'Total'},inplace=True)
Orders_df=Orders_df.sort_values(by='PONumber')

In [30]:
Orders_df["OrderID"]=range(1,len(Orders_df)+1)
Orders_df=Orders_df[["OrderID","PODate_ID","InvoiceDate_ID","PayDate_ID","PONumber","SupplierID","Quantity","Total","Freight"]]

In [31]:
Orders_df

,OrderID,PODate_ID,InvoiceDate_ID,PayDate_ID,PONumber,SupplierID,Quantity,Total,Freight
3,1,1,34,82,8106,8,10100,137483.78,2935.20
10,2,1,20,92,8107,116,24,348.72,9.08
14,3,1,32,92,8108,17,8466,60281.13,1549.81
17,4,1,34,94,8109,22,2246,14298.09,408.72
22,5,1,28,110,8110,37,8086,56493.23,1300.92
...,...,...,...,...,...,...,...,...,...
5526,5539,429,446,487,13657,128,65,1248.61,5.74
5532,5540,429,447,487,13658,84,38896,502128.05,2761.70
5533,5541,429,441,478,13659,107,195,2599.66,11.70
5537,5542,429,444,487,13660,122,60,694.68,3.20


*After it Fully   Normalized We can Now Populate Our Fact_Order Table in Our database*

# Fact_OrderItems

*After Designing The Scheme*

*The Required Design SHould Be Carfully Aborachad*

*This Fact Table Should Contains The Details Of Each Orders*

In [32]:
Orders_items_df=normalized_dataframes['PurchasesFINAL12312016.csv']


In [33]:
Orders_items_df=Orders_items_df.sort_values(by="PONumber")
Orders_items_df=pd.merge(Orders_items_df,Orders_df[['OrderID','PONumber']],on='PONumber',how='right')

In [34]:
Orders_items_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,OrderID
0,54_GUTHRAM_2663,54,2663,Dewars White Label,1.75L,480,BACARDI USA INC,8106,21.42,12,257.04,1,1,14,34,82,1
1,30_CULCHETH_3961,30,3961,Grey Goose La Poire Vodka,750mL,480,BACARDI USA INC,8106,18.17,12,218.04,1,1,12,34,82,1
2,30_CULCHETH_2663,30,2663,Dewars White Label,1.75L,480,BACARDI USA INC,8106,21.42,10,214.20,1,1,12,34,82,1
3,38_GOULCREST_7990,38,7990,Noilly Prat Sweet Vermouth,Liter,480,BACARDI USA INC,8106,7.68,12,92.16,1,1,12,34,82,1
4,38_GOULCREST_4881,38,4881,Bacardi Twin Pack 2/750mls,750mL 2 Pk,480,BACARDI USA INC,8106,14.81,6,88.86,1,1,12,34,82,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,78_EASTHAVEN_23095,78,23095,Glen Ellen Cab Svgn RSV,1.5L,9815,WINE GROUP INC,13661,4.76,60,285.60,2,429,436,446,476,5543
2372470,78_EASTHAVEN_23095,78,23095,Glen Ellen Cab Svgn RSV,1.5L,9815,WINE GROUP INC,13661,4.76,6,28.56,2,429,432,446,476,5543
2372471,60_IRRAGIN_37904,60,37904,Trapiche Broquel Mndz Malbec,750mL,9815,WINE GROUP INC,13661,6.45,12,77.40,2,429,432,446,476,5543
2372472,68_SOLARIS_11828,68,11828,Cupcake Svgn Bl,750mL,9815,WINE GROUP INC,13661,3.97,6,23.82,2,429,433,446,476,5543


In [35]:
Orders_items_df=pd.merge(Orders_items_df,Suppliers,on=['VendorNumber',"VendorName"],how='right')
Orders_items_df.rename(columns={'ID':'SupplierID'},inplace=True)


In [36]:
Orders_items_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,OrderID,SupplierID
0,73_DONCASTER_90085,73,90085,Ch Lilian 09 Ladouys St Este,750mL,2,"IRA GOLDMAN AND WILLIAMS, LLP",8848,23.86,5,119.30,2,90,108,116,172,731,1
1,79_BALLYMENA_90609,79,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,2,181,191,201,231,1738,1
2,67_EANVERNESS_90609,67,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,2,181,191,201,231,1738,1
3,73_DONCASTER_90609,73,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,2,181,191,201,231,1738,1
4,76_DONCASTER_90609,76,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,2,181,191,201,231,1738,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,23_ARBINGTON_3909,23,3909,White Mountain Vodka,750mL,173357,TAMWORTH DISTILLING,13588,19.37,12,232.44,1,425,435,442,486,5470,128
2372470,39_EASTHALLOW_3666,39,3666,Art in the Age Chicory Root,375mL,173357,TAMWORTH DISTILLING,13588,18.79,12,225.48,1,425,432,442,486,5470,128
2372471,42_BLACK HOLLOW_3909,42,3909,White Mountain Vodka,750mL,173357,TAMWORTH DISTILLING,13588,19.37,24,464.88,1,425,433,442,486,5470,128
2372472,23_ARBINGTON_3666,23,3666,Art in the Age Chicory Root,375mL,173357,TAMWORTH DISTILLING,13588,18.79,6,112.74,1,425,433,442,486,5470,128


In [37]:
Orders_items_df=pd.merge(Orders_items_df,Products[['ProductID','Description','Size','PurchasePrice','SupplierID']],on=['Description','Size','PurchasePrice','SupplierID'],how='right')

In [38]:
Orders_items_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,OrderID,SupplierID,ProductID
0,1_HARDERSFIELD_58,1,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8166,9.28,6,55.68,1,4,12,26,94,54,75,1
1,50_MOUNTMEND_58,50,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8166,9.28,5,46.40,1,4,14,26,94,54,75,1
2,49_GARIGILL_58,49,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8246,9.28,6,55.68,1,9,26,48,114,129,75,1
3,79_BALLYMENA_58,79,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8246,9.28,11,102.08,1,9,28,48,114,129,75,1
4,32_MOUNTMEND_58,32,58,Gekkeikan Black & Gold Sake,750mL,8320,SHAW ROSS INT L IMP LTD,8246,9.28,6,55.68,1,9,26,48,114,129,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,41_LARNWICK_90631,41,90631,Luxardo Maraschino Cherries,400mL,9165,ULTRA BEVERAGE COMPANY LLP,13462,12.74,12,152.88,2,417,429,433,465,5344,84,10699
2372470,69_MOUNTMEND_90631,69,90631,Luxardo Maraschino Cherries,400mL,9165,ULTRA BEVERAGE COMPANY LLP,13462,12.74,9,114.66,2,417,428,433,465,5344,84,10699
2372471,67_EANVERNESS_90631,67,90631,Luxardo Maraschino Cherries,400mL,9165,ULTRA BEVERAGE COMPANY LLP,13462,12.74,9,114.66,2,417,429,433,465,5344,84,10699
2372472,38_GOULCREST_90631,38,90631,Luxardo Maraschino Cherries,400mL,9165,ULTRA BEVERAGE COMPANY LLP,13462,12.74,12,152.88,2,417,429,433,465,5344,84,10699


In [39]:
Orders_items_df=Orders_items_df.sort_values(by=['OrderID','PODate_ID'])
Orders_items_df["ItemsID"]=range(1,len(Orders_items_df)+1)

In [40]:
Orders_items_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,OrderID,SupplierID,ProductID,ItemsID
5014,60_IRRAGIN_126,60,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,3,60.42,1,1,12,34,82,1,8,28,1
5015,1_HARDERSFIELD_126,1,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,12,241.68,1,1,12,34,82,1,8,28,2
5016,34_PITMERDEN_126,34,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,6,120.84,1,1,14,34,82,1,8,28,3
5017,66_EANVERNESS_126,66,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,4,80.56,1,1,12,34,82,1,8,28,4
5018,47_PELLA'S WISH_126,47,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,3,60.42,1,1,14,34,82,1,8,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2314478,11_CARDEND_43450,11,43450,Fish Eye Pnt Grigio,3L,9815,WINE GROUP INC,13661,11.33,6,67.98,2,429,435,446,476,5543,92,10282,2372470
2314479,69_MOUNTMEND_43450,69,43450,Fish Eye Pnt Grigio,3L,9815,WINE GROUP INC,13661,11.33,6,67.98,2,429,436,446,476,5543,92,10282,2372471
2349651,23_ARBINGTON_45632,23,45632,Fish Eye Chard Cal,3L,9815,WINE GROUP INC,13661,11.10,6,66.60,2,429,435,446,476,5543,92,10525,2372472
2355654,50_MOUNTMEND_46065,50,46065,Corbett Canyon Pnt Nr,3L,9815,WINE GROUP INC,13661,8.44,6,50.64,2,429,432,446,476,5543,92,10565,2372473


In [41]:
Orders_items_df=pd.merge(Orders_items_df,Stores[['ID','Store']],on='Store',how='left')
Orders_items_df.rename(columns={'ID':"StoreID"},inplace=True)
Orders_items_df=Orders_items_df[["ItemsID","OrderID","ProductID","SupplierID","StoreID","Quantity","PurchasePrice","Dollars"]]

In [42]:
Orders_items_df

,ItemsID,OrderID,ProductID,SupplierID,StoreID,Quantity,PurchasePrice,Dollars
0,1,1,28,8,60,3,20.14,60.42
1,2,1,28,8,1,12,20.14,241.68
2,3,1,28,8,34,6,20.14,120.84
3,4,1,28,8,66,4,20.14,80.56
4,5,1,28,8,47,3,20.14,60.42
...,...,...,...,...,...,...,...,...
2372469,2372470,5543,10282,92,11,6,11.33,67.98
2372470,2372471,5543,10282,92,69,6,11.33,67.98
2372471,2372472,5543,10525,92,23,6,11.10,66.60
2372472,2372473,5543,10565,92,50,6,8.44,50.64


COPY persons (id, name, age)

FROM '/path/to/data.csv'

DELIMITER ','

CSV HEADER;




# Creating Fact_Inventory 

In [430]:
inventory_df=normalized_dataframes['PurchasesFINAL12312016.csv']


In [431]:
inventory_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,35.71,6,214.26,Non-Alchloic,2,14,18,104
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,9.35,4,37.40,Non-Alchloic,3,12,24,114
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,9.41,5,47.05,Non-Alchloic,3,14,24,114
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,9.35,6,56.10,Non-Alchloic,3,12,24,114
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,21.32,5,106.60,Non-Alchloic,5,14,28,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_GARIGILL_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,8.06,12,96.72,Non-Alchloic,425,434,446,474
2372470,1_HARDERSFIELD_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,9.39,12,112.68,Non-Alchloic,425,433,446,474
2372471,66_EANVERNESS_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,6.75,12,81.00,Non-Alchloic,425,432,446,474
2372472,69_MOUNTMEND_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,9.39,12,112.68,Non-Alchloic,425,432,446,474


In [432]:
inventory_df=inventory_df.sort_values(by='PONumber')
inventory_df=pd.merge(inventory_df,Suppliers,on=['VendorNumber',"VendorName"],how='right')
inventory_df.rename(columns={'ID':'SupplierID'},inplace=True)

In [433]:
inventory_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,SupplierID
0,73_DONCASTER_90085,73,90085,Ch Lilian 09 Ladouys St Este,750mL,2,"IRA GOLDMAN AND WILLIAMS, LLP",8848,23.86,5,119.30,Non-Alchloic,90,108,116,172,1
1,79_BALLYMENA_90609,79,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,Non-Alchloic,181,191,201,231,1
2,67_EANVERNESS_90609,67,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,Non-Alchloic,181,191,201,231,1
3,73_DONCASTER_90609,73,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,Non-Alchloic,181,191,201,231,1
4,76_DONCASTER_90609,76,90609,Flavor Essence Variety 5 Pak,5/2 oz,2,"IRA GOLDMAN AND WILLIAMS, LLP",9855,17.00,32,544.00,Non-Alchloic,181,191,201,231,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,23_ARBINGTON_3909,23,3909,White Mountain Vodka,750mL,173357,TAMWORTH DISTILLING,13588,19.37,12,232.44,Non-Alchloic,425,435,442,486,128
2372470,39_EASTHALLOW_3666,39,3666,Art in the Age Chicory Root,375mL,173357,TAMWORTH DISTILLING,13588,18.79,12,225.48,Non-Alchloic,425,432,442,486,128
2372471,42_BLACK HOLLOW_3909,42,3909,White Mountain Vodka,750mL,173357,TAMWORTH DISTILLING,13588,19.37,24,464.88,Non-Alchloic,425,433,442,486,128
2372472,23_ARBINGTON_3666,23,3666,Art in the Age Chicory Root,375mL,173357,TAMWORTH DISTILLING,13588,18.79,6,112.74,Non-Alchloic,425,433,442,486,128


In [434]:
inventory_df=pd.merge(inventory_df,Products[['ProductID','Description','Size','PurchasePrice','SupplierID']],on=['Description','Size','PurchasePrice','SupplierID'],how='right')
inventory_df=inventory_df.sort_values(by=['PONumber','ReceivingDate_ID'])

In [435]:
inventory_df

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PurchasePrice,Quantity,Dollars,Classification,PODate_ID,ReceivingDate_ID,InvoiceDate_ID,PayDate_ID,SupplierID,ProductID
5014,60_IRRAGIN_126,60,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,3,60.42,Non-Alchloic,1,12,34,82,8,28
5015,1_HARDERSFIELD_126,1,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,12,241.68,Non-Alchloic,1,12,34,82,8,28
5017,66_EANVERNESS_126,66,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,4,80.56,Non-Alchloic,1,12,34,82,8,28
5020,30_CULCHETH_126,30,126,Grey Goose Vodka,Liter,480,BACARDI USA INC,8106,20.14,12,241.68,Non-Alchloic,1,12,34,82,8,28
23974,1_HARDERSFIELD_443,1,443,St Germain Liqueur,750mL,480,BACARDI USA INC,8106,24.45,6,146.70,Non-Alchloic,1,12,34,82,8,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225809,1_HARDERSFIELD_39087,1,39087,Corbett Canyon Pnt Grigio,3L,9815,WINE GROUP INC,13661,6.66,3,19.98,Non-Alchloic,429,437,446,476,92,9864
2247949,6_GOULCREST_40051,6,40051,Concannon Select Vyds Pnt Nr,750mL,9815,WINE GROUP INC,13661,4.54,12,54.48,Non-Alchloic,429,437,446,476,92,9967
2283105,1_HARDERSFIELD_41711,1,41711,Fish Eye Cab Svgn,750mL,9815,WINE GROUP INC,13661,3.22,12,38.64,Non-Alchloic,429,437,446,476,92,10101
2284700,1_HARDERSFIELD_41713,1,41713,Fish Eye Merlot,750mL,9815,WINE GROUP INC,13661,3.22,9,28.98,Non-Alchloic,429,437,446,476,92,10103


In [436]:
inventory_df=pd.merge(inventory_df,Stores[['ID','Store']],on='Store',how='left')
inventory_df.rename(columns={'ID':"StoreID"},inplace=True)
inventory_df=inventory_df[["ReceivingDate_ID","StoreID","ProductID","Quantity","PONumber","Dollars","InventoryId"]]

In [437]:
inventory_df

,ReceivingDate_ID,StoreID,ProductID,Quantity,PONumber,Dollars,InventoryId
0,12,60,28,3,8106,60.42,60_IRRAGIN_126
1,12,1,28,12,8106,241.68,1_HARDERSFIELD_126
2,12,66,28,4,8106,80.56,66_EANVERNESS_126
3,12,30,28,12,8106,241.68,30_CULCHETH_126
4,12,1,169,6,8106,146.70,1_HARDERSFIELD_443
...,...,...,...,...,...,...,...
2372469,437,1,9864,3,13661,19.98,1_HARDERSFIELD_39087
2372470,437,6,9967,12,13661,54.48,6_GOULCREST_40051
2372471,437,1,10101,12,13661,38.64,1_HARDERSFIELD_41711
2372472,437,1,10103,9,13661,28.98,1_HARDERSFIELD_41713


In [438]:
inventory_df["InventoryID"]=range(1,len(inventory_df)+1)

In [440]:
inventory_df=inventory_df[["InventoryID","ReceivingDate_ID","StoreID","ProductID","Quantity","PONumber","Dollars","InventoryId"]]

In [441]:
inventory_df

,InventoryID,ReceivingDate_ID,StoreID,ProductID,Quantity,PONumber,Dollars,InventoryId
0,1,12,60,28,3,8106,60.42,60_IRRAGIN_126
1,2,12,1,28,12,8106,241.68,1_HARDERSFIELD_126
2,3,12,66,28,4,8106,80.56,66_EANVERNESS_126
3,4,12,30,28,12,8106,241.68,30_CULCHETH_126
4,5,12,1,169,6,8106,146.70,1_HARDERSFIELD_443
...,...,...,...,...,...,...,...,...
2372469,2372470,437,1,9864,3,13661,19.98,1_HARDERSFIELD_39087
2372470,2372471,437,6,9967,12,13661,54.48,6_GOULCREST_40051
2372471,2372472,437,1,10101,12,13661,38.64,1_HARDERSFIELD_41711
2372472,2372473,437,1,10103,9,13661,28.98,1_HARDERSFIELD_41713


# Creating Fact_Sales Table

1. *We Start to Creating the Fact Table*

In [503]:
Sales_df=normalized_dataframes['SalesFINAL12312016.csv']

In [504]:
Sales_df

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNumber,VendorName,SalesDate_ID
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,750,1,0.79,12546,JIM BEAM BRANDS COMPANY,12
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,750,1,1.57,12546,JIM BEAM BRANDS COMPANY,14
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,750,1,0.79,12546,JIM BEAM BRANDS COMPANY,16
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,750,1,0.79,12546,JIM BEAM BRANDS COMPANY,26
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,375,1,0.79,12546,JIM BEAM BRANDS COMPANY,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,4,51.80,12.95,750,2,0.45,2000,SOUTHERN WINE & SPIRITS NE,96
1048571,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,8,103.60,12.95,750,2,0.90,2000,SOUTHERN WINE & SPIRITS NE,98
1048572,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,1,12.95,12.95,750,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,110
1048573,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,3,38.85,12.95,750,2,0.34,2000,SOUTHERN WINE & SPIRITS NE,116


In [505]:
inventory_df_normalized=inventory_df.drop_duplicates(subset='InventoryId')
Sales_df=pd.merge(Sales_df,inventory_df_normalized,on="InventoryId",how='left')
Sales_df

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,...,VendorNumber,VendorName,SalesDate_ID,InventoryID,ReceivingDate_ID,StoreID,ProductID,Quantity,PONumber,Dollars
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,750,1,...,12546,JIM BEAM BRANDS COMPANY,12,1739651.0,346.0,1.0,464.0,18.0,12271.0,191.70
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,750,1,...,12546,JIM BEAM BRANDS COMPANY,14,1739651.0,346.0,1.0,464.0,18.0,12271.0,191.70
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,750,1,...,12546,JIM BEAM BRANDS COMPANY,16,1739651.0,346.0,1.0,464.0,18.0,12271.0,191.70
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,750,1,...,12546,JIM BEAM BRANDS COMPANY,26,1739651.0,346.0,1.0,464.0,18.0,12271.0,191.70
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,375,1,...,12546,JIM BEAM BRANDS COMPANY,28,2481.0,12.0,1.0,465.0,6.0,8142.0,164.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,4,51.80,12.95,750,2,...,2000,SOUTHERN WINE & SPIRITS NE,96,99138.0,52.0,19.0,9888.0,48.0,8450.0,430.08
1048571,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,8,103.60,12.95,750,2,...,2000,SOUTHERN WINE & SPIRITS NE,98,99138.0,52.0,19.0,9888.0,48.0,8450.0,430.08
1048572,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,1,12.95,12.95,750,2,...,2000,SOUTHERN WINE & SPIRITS NE,110,99138.0,52.0,19.0,9888.0,48.0,8450.0,430.08
1048573,19_WINTERVALE_39384,19,39384,F Coppola Diamond Pnt Nr,750mL,3,38.85,12.95,750,2,...,2000,SOUTHERN WINE & SPIRITS NE,116,99138.0,52.0,19.0,9888.0,48.0,8450.0,430.08


In [506]:
Sales_df = Sales_df.dropna()
Sales_df=Sales_df.sort_values(by=["SalesDate_ID","ReceivingDate_ID"])
Sales_df["SalesID"]=range(1,len(Sales_df)+1)
Sales_df=Sales_df[["SalesID","SalesDate_ID","StoreID","ProductID","InventoryID","SalesQuantity","SalesDollars","ExciseTax"]]
Sales_df["InventoryID"]=Sales_df["InventoryID"].astype(int)
Sales_df["StoreID"]=Sales_df["StoreID"].astype(int)
Sales_df["ProductID"]=Sales_df["ProductID"].astype(int)
Sales_df["SalesQuantity"]=Sales_df["SalesQuantity"].astype(int)
Sales_df

,SalesID,SalesDate_ID,StoreID,ProductID,InventoryID,SalesQuantity,SalesDollars,ExciseTax
106540,1,12,18,4376,4893,1,8.95,0.22
106772,2,12,18,890,603,1,12.99,1.84
106828,3,12,18,903,7545,4,41.96,7.35
107217,4,12,18,9137,7100,1,17.99,0.75
107335,5,12,18,1550,7321,1,8.79,1.84
...,...,...,...,...,...,...,...,...
1022471,1008514,130,15,2167,1986198,1,34.99,0.79
990950,1008515,130,13,3581,1979461,1,14.99,0.11
1037780,1008516,130,17,9851,1992073,1,11.99,0.11
1031051,1008517,130,16,1691,2043941,1,1.99,0.05


In [507]:
Sales_df